<a href="https://colab.research.google.com/github/racoope70/exploratory_daytrading/blob/main/multi_stock_sac_inference_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Fix Potential Library Conflicts
!apt-get remove --purge -y cuda* libcuda* nvidia* || echo "No conflicting CUDA packages"
!apt-get autoremove -y
!apt-get clean

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'cuda-toolkit-12-4-config-common' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-11-0' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-11-1' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-11-7' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-11-8' for glob 'cuda*'
Note, selecting 'cuda-toolkit-12-5-config-common' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-0' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-1' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-2' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-3' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-4' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-5' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-6' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-8' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-12-9' for glob 'cuda*'
Note, selecting 'cuda-cudart-dev-13-0' fo

In [ ]:
#Protocol Buffer Fix (for TensorFlow)
!pip uninstall -y protobuf
!pip install protobuf==3.20.3

Found existing installation: protobuf 5.29.5
Uninstalling protobuf-5.29.5:
  Successfully uninstalled protobuf-5.29.5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 3.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.71.2 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 3.20.3 which is incompatible.
tensorflow-metadata 1.17.2 requires protobuf>=4.25.2; python_version >= "3.11", but you have protobuf 3.20.3 which is incompatible.
ydf 0.13.0 requires protobuf<7.0.0,>=5.29.1, but you have protobuf 3.20.3 which is incompatible.


In [ ]:
#Update Colab Environment and System Libraries
!apt-get update -y && apt-get upgrade -y


Hit:1 https://cli.github.com/packages stable InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Ign:11 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Ign:11 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Ign:11 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Err:11 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
  Could not connect to r2u.stat.illinois.edu:443 (192

In [ ]:
#Install Correct Version of CUDA for Colab GPU
!apt-get update -qq && apt-get install -y \
    libcusolver11 libcusparse11 libcurand10 libcufft10 libnppig10 libnppc10 libnppial10 \
    cuda-toolkit-12-4

W: Failed to fetch https://r2u.stat.illinois.edu/ubuntu/dists/jammy/InRelease  Could not connect to r2u.stat.illinois.edu:443 (192.17.190.167), connection timed out
W: Some index files failed to download. They have been ignored, or old ones used instead.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package libnppig10
E: Unable to locate package libnppc10
E: Unable to locate package libnppial10


In [ ]:
#Set Correct CUDA Paths
import os
os.environ['CUDA_HOME'] = '/usr/local/cuda-12.4'
os.environ['PATH'] += ':/usr/local/cuda-12.4/bin'
os.environ['LD_LIBRARY_PATH'] += ':/usr/local/cuda-12.4/lib64'


In [ ]:
#Install RAPIDS and NVIDIA Dependencies
!pip install --extra-index-url=https://pypi.nvidia.com \
    cuml-cu12==25.2.0 cudf-cu12==25.2.0 cupy-cuda12x dask-cuda==25.2.0 dask-cudf-cu12==25.2.0


Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com


In [ ]:
#Install TensorFlow (latest GPU-compatible version)
!pip install tensorflow==2.18.0

#Install Stable Baselines3 and Trading Libraries
!pip install stable-baselines3[extra] gymnasium gym-anytrading yfinance xgboost joblib

#Install Miscellaneous Libraries
!pip install matplotlib scikit-learn pandas numba==0.61.0

#Install PyTorch with GPU Support
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124


  Using cached tensorflow-2.18.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.1 kB)
Using cached tensorflow-2.18.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (615.5 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.19.0 requires tensorflow<2.20,>=2.19, but you have tensorflow 2.18.0 which is incompatible.
tensorflow-decision-forests 1.12.0 requires tensorflow==2.19.0, but you have tensorflow 2.18.0 which is incompatible.
tensorflow-text 2.19.0 requires tensorflow<2.20,>=2.19.0, but you have tensorflow 2.18.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.2/172.2 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.2/187.2 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
#Install TensorFlow (latest GPU-compatible version)
!pip install tensorflow==2.18.0

#Immediately follow it with:
import tensorflow as tf

gpus = tf.config.list_physical_devices("GPU")
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("TensorFlow GPU memory growth enabled")
    except RuntimeError as e:
        print(f"TensorFlow GPU memory config failed: {e}")


  Using cached tensorflow-2.18.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.1 kB)
Using cached tensorflow-2.18.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (615.5 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.19.0 requires tensorflow<2.20,>=2.19, but you have tensorflow 2.18.0 which is incompatible.
tensorflow-decision-forests 1.12.0 requires tensorflow==2.19.0, but you have tensorflow 2.18.0 which is incompatible.
tensorflow-text 2.19.0 requires tensorflow<2.20,>=2.19.0, but you have tensorflow 2.18.0 which is incompatible.


In [ ]:
!fusermount -u /content/drive

fusermount: failed to unmount /content/drive: No such file or directory


In [ ]:
from google.colab import drive
drive.flush_and_unmount()  #  safely unmounts


Drive not mounted, so nothing to flush and unmount.


In [ ]:
# ---------------- Imports (local to this section) ----------------
import os, gc, time, logging, warnings, sys, re
from datetime import datetime
import numpy as np
import pandas as pd
import yfinance as yf
import pywt

warnings.filterwarnings("ignore", category=FutureWarning)

# ---------------- Paths ----------------
BASE_DIR   = "./sac_parity"
DATA_DIR   = os.path.join(BASE_DIR, "data")
LOG_DIR    = os.path.join(BASE_DIR, "logs")
os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(LOG_DIR, exist_ok=True)

FEATURE_CSV = "multi_stock_feature_engineered_dataset.csv"  # combined output used by training

# ---------------- Logger (merged) ----------------
def setup_logger(name="download_fe", level=logging.INFO):
    logger = logging.getLogger(name)
    logger.setLevel(level)
    # Avoid duplicate handlers if this cell/file runs more than once
    logger.handlers = []
    fmt = logging.Formatter("%(asctime)s | %(levelname)s | %(name)s | %(message)s")

    sh = logging.StreamHandler(sys.stdout)
    sh.setFormatter(fmt); sh.setLevel(level)
    logger.addHandler(sh)

    fh = logging.FileHandler(os.path.join(LOG_DIR, f"{name}.log"))
    fh.setFormatter(fmt); fh.setLevel(level)
    logger.addHandler(fh)
    return logger

log = setup_logger("download_fe")

# ---------------- Feature Toggles (PPO-style) ----------------
USE_SENTIMENT  = False   # placeholder (constant 0.0 if False)
USE_REGIME     = True
USE_WAVELET    = True
USE_GREEKS     = True

# ---------------- Data Params ----------------
INTERVAL    = "1h"
PERIOD_DAYS = 720

# ---------------- Universe + HP Buckets (from 'Download 2') ----------------
TICKERS = [
    'AAPL','TSLA','MSFT','GOOGL','AMZN','NVDA','META','BRK-B','JPM','JNJ',
    'XOM','V','PG','UNH','MA','HD','LLY','MRK','PEP','KO',
    'BAC','ABBV','AVGO','PFE','COST','CSCO','TMO','ABT','ACN','WMT',
    'MCD','ADBE','DHR','CRM','NKE','INTC','QCOM','NEE','AMD','TXN',
    'AMGN','UPS','LIN','PM','UNP','BMY','LOW','RTX','CVX','IBM',
    'GE','SBUX','ORCL'
]
SYMBOLS = TICKERS  # override elsewhere if needed (e.g., short tests)

FAST_NAMES = {
    "TSLA","NVDA","AMD","AVGO","AAPL","MSFT","AMZN","GOOGL","META","ADBE","CRM",
    "INTC","QCOM","TXN","ORCL","NEE","GE","XOM","CVX","LLY","NKE","SBUX"
}
SLOW_NAMES = {
    "BRK-B","JPM","BAC","JNJ","UNH","MRK","PFE","ABBV","ABT","AMGN","PG","PEP","KO",
    "V","MA","WMT","MCD","TMO","DHR","ACN","IBM","LIN","PM","RTX","UPS","UNP","COST","HD","LOW"
}

def pick_bucket(symbol: str) -> str:
    """Return 'FAST', 'SLOW', or 'MED' — handy for downstream training configs."""
    if symbol in FAST_NAMES: return "FAST"
    if symbol in SLOW_NAMES: return "SLOW"
    return "MED"

# ---------------- Helpers: schema + datetime ----------------
def _force_datetime_column(df: pd.DataFrame) -> pd.DataFrame:
    """Ensure tz-naive 'Datetime' exists, dedup and sort."""
    if isinstance(df.index, pd.DatetimeIndex):
        try:
            df.index = df.index.tz_convert(None)
        except Exception:
            try:
                df.index = df.index.tz_localize(None)
            except Exception:
                pass
        df.index.name = 'Datetime'
        df = df.reset_index()
    else:
        df = df.reset_index()
        first = df.columns[0]
        if np.issubdtype(df[first].dtype, np.datetime64):
            df = df.rename(columns={first: 'Datetime'})
        elif 'Date' in df.columns:
            df['Datetime'] = pd.to_datetime(df['Date'])
        elif 'Datetime' not in df.columns:
            df['Datetime'] = pd.to_datetime(df[first], errors='coerce')

    if 'Datetime' not in df.columns:
        raise KeyError("Failed to construct 'Datetime' from data.")
    df['Datetime'] = pd.to_datetime(df['Datetime'])
    return df.drop_duplicates(subset=['Datetime']).sort_values('Datetime').reset_index(drop=True)

def _normalize_ohlcv(df_in: pd.DataFrame, ticker: str) -> pd.DataFrame:
    """Flatten MultiIndex, strip ticker prefixes/suffixes, map to canonical OHLCV names."""
    df = df_in.copy()
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = [" ".join([str(p) for p in col if p]) for col in df.columns]
    df.columns = [re.sub(r"\s+", " ", str(c)).strip() for c in df.columns]

    tkr = ticker.upper().replace("-", "[- ]?")
    cleaned = {}
    for c in df.columns:
        cu = c.upper()
        cu = re.sub(rf"^(?:{tkr})[\s/_-]+", "", cu)
        cu = re.sub(rf"[\s/_-]+(?:{tkr})$", "", cu)
        cleaned[c] = cu.title()
    if any(cleaned[c] != c for c in df.columns):
        df = df.rename(columns=cleaned)

    cols_ci = {c.lower(): c for c in df.columns}
    wants = {
        "Open":      ["open"],
        "High":      ["high"],
        "Low":       ["low"],
        "Close":     ["close","last","close*"],
        "Adj Close": ["adj close","adj_close","adjclose","adjusted close"],
        "Volume":    ["volume","vol"]
    }
    rename_map = {}
    for desired, alts in wants.items():
        if desired.lower() in cols_ci:
            rename_map[cols_ci[desired.lower()]] = desired
        else:
            for a in alts:
                if a in cols_ci:
                    rename_map[cols_ci[a]] = desired
                    break
    if rename_map:
        df = df.rename(columns=rename_map)
    return df

# ---------------- Downloader (retry + history() fallback) ----------------
def download_stock_data(ticker, interval="1h", period_days=720, max_retries=5, sleep_base=3):
    """
    Robust yfinance intraday downloader.
    Guarantees: Open, High, Low, Close, Volume (+Adj Close), Datetime, Symbol.
    """
    period_str = f"{int(period_days)}d"

    def _post(df: pd.DataFrame) -> pd.DataFrame:
        df = _normalize_ohlcv(df, ticker)
        df = _force_datetime_column(df)
        needed = {'Open', 'High', 'Low', 'Close', 'Volume'}
        missing = needed - set(df.columns)
        if missing:
            raise ValueError(f"Missing OHLCV columns after normalize: {missing}")
        if 'Adj Close' not in df.columns:
            df['Adj Close'] = df['Close']
        return df

    for attempt in range(1, max_retries + 1):
        try:
            log.info(f"[{ticker}] Attempt {attempt}: download({period_str}, {interval})")
            df = yf.download(
                tickers=ticker,
                period=period_str,
                interval=interval,
                progress=False,
                auto_adjust=False,
                group_by='column',
                threads=False,
                prepost=False,
                repair=True
            )
            if df is None or df.empty:
                raise ValueError("Empty data from download()")
            df = _post(df)
            df['Symbol'] = ticker
            log.info(f"[{ticker}] rows={len(df)} {df['Datetime'].min()}→{df['Datetime'].max()}")
            return df

        except Exception as e1:
            log.warning(f"[{ticker}] download error: {e1} | trying history()")
            try:
                hist = yf.Ticker(ticker).history(
                    period=period_str,
                    interval=interval,
                    auto_adjust=False,
                    actions=False
                )
                if hist is None or hist.empty:
                    raise ValueError("Empty data from history()")
                df = _post(hist)
                df['Symbol'] = ticker
                log.info(f"[{ticker}] (fallback) rows={len(df)} {df['Datetime'].min()}→{df['Datetime'].max()}")
                return df
            except Exception as e2:
                wait = sleep_base * attempt
                log.warning(f"[{ticker}] history() error: {e2} | retrying in {wait}s")
                time.sleep(wait)

    log.error(f"[{ticker}] Failed after {max_retries} attempts.")
    return None

# ---------------- Feature Engineering (parity) ----------------
def denoise_wavelet(series, wavelet='db1', level=2):
    s = pd.Series(series).astype(float).ffill().bfill().to_numpy()
    try:
        coeffs = pywt.wavedec(s, wavelet, mode='symmetric', level=level)
        # hard smoothing: zero high-frequency detail
        for i in range(1, len(coeffs)):
            coeffs[i] = np.zeros_like(coeffs[i])
        rec = pywt.waverec(coeffs, wavelet, mode='symmetric')
        return pd.Series(rec[:len(s)], index=series.index)
    except Exception as e:
        log.warning(f"Wavelet denoise failed ({e}); returning raw series.")
        return pd.Series(s, index=series.index)

def add_regime(df: pd.DataFrame) -> pd.DataFrame:
    df['Vol20'] = df['Close'].pct_change().rolling(20).std()
    df['Ret20'] = df['Close'].pct_change(20)
    vol_hi   = (df['Vol20'] > df['Vol20'].median()).astype(int)
    trend_hi = (df['Ret20'].abs() > df['Ret20'].abs().median()).astype(int)
    df['Regime4'] = vol_hi * 2 + trend_hi  # 0..3
    return df

def compute_enhanced_features(df: pd.DataFrame):
    """
    Returns (feature_df, FEATURES) with PPO-parity indicators for SAC.
    """
    df = df.copy()
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.get_level_values(0)
    df = df.loc[:, ~df.columns.duplicated()]

    # Core techs
    df['SMA_20'] = df['Close'].rolling(20).mean()
    df['STD_20'] = df['Close'].rolling(20).std()

    # Bollinger bands
    df['Upper_Band'] = df['SMA_20'] + 2 * df['STD_20']
    df['Lower_Band'] = df['SMA_20'] - 2 * df['STD_20']

    # Stochastic %K
    df['Lowest_Low']   = df['Low'].rolling(14).min()
    df['Highest_High'] = df['High'].rolling(14).max()
    denom = (df['Highest_High'] - df['Lowest_Low']).replace(0, np.nan)
    df['Stoch'] = ((df['Close'] - df['Lowest_Low']) / denom) * 100

    df['ROC'] = df['Close'].pct_change(10)
    df['OBV'] = (np.sign(df['Close'].diff()).fillna(0) * df['Volume'].fillna(0)).cumsum()

    # CCI
    tp = (df['High'] + df['Low'] + df['Close']) / 3
    sma_tp = tp.rolling(20).mean()
    md = (tp - sma_tp).abs().rolling(20).mean()
    df['CCI'] = (tp - sma_tp) / (0.015 * md)

    # EMAs & MACD
    df['EMA_10'] = df['Close'].ewm(span=10, adjust=False).mean()
    df['EMA_50'] = df['Close'].ewm(span=50, adjust=False).mean()
    ema12 = df['Close'].ewm(span=12, adjust=False).mean()
    ema26 = df['Close'].ewm(span=26, adjust=False).mean()
    df['MACD_Line']   = ema12 - ema26
    df['MACD_Signal'] = df['MACD_Line'].ewm(span=9, adjust=False).mean()

    # RSI
    delta = df['Close'].diff()
    gain  = delta.clip(lower=0).rolling(14).mean()
    loss  = (-delta.clip(upper=0)).rolling(14).mean()
    rs = gain / (loss.replace(0, np.nan))
    df['RSI'] = 100 - (100 / (1 + rs))

    # ATR
    tr = pd.concat([
        (df['High'] - df['Low']),
        (df['High'] - df['Close'].shift()).abs(),
        (df['Low'] - df['Close'].shift()).abs()
    ], axis=1).max(axis=1)
    df['ATR'] = tr.rolling(14).mean()

    df['Volatility'] = df['Close'].pct_change().rolling(20).std()

    # Denoised close
    df['Denoised_Close'] = denoise_wavelet(df['Close'].ffill()) if USE_WAVELET else df['Close']

    # Regime
    if USE_REGIME:
        df = add_regime(df)

    # Sentiment placeholder (0.0 if disabled)
    df['SentimentScore'] = 0.0 if not USE_SENTIMENT else df.get('SentimentScore', 0.0)

    # Greeks proxies
    if USE_GREEKS:
        df['Delta']  = df['Close'].pct_change(1).fillna(0)
        df['Gamma']  = df['Delta'].diff().fillna(0)
        df['Delta1'] = df['Denoised_Close'].pct_change(1).fillna(0)
        df['Gamma1'] = df['Delta1'].diff().fillna(0)

    # Cleanup
    df = df.dropna().reset_index(drop=True)

    FEATURES = [
        'SMA_20','STD_20','Upper_Band','Lower_Band','Stoch','ROC','OBV','CCI',
        'EMA_10','EMA_50','MACD_Line','MACD_Signal','RSI','ATR','Volatility',
        'Denoised_Close','Regime4','SentimentScore','Delta','Gamma','Delta1','Gamma1'
    ]
    base_cols = ['Datetime','Open','High','Low','Close','Adj Close','Volume','Symbol']
    keep = [c for c in base_cols if c in df.columns] + [f for f in FEATURES if f in df.columns]
    df = df[keep]
    df['Datetime'] = pd.to_datetime(df['Datetime']).dt.tz_localize(None)
    return df, FEATURES

# ---------------- Orchestrator ----------------
def build_features_if_needed() -> pd.DataFrame:
    """
    If FEATURE_CSV exists, load and return it (no download).
    Else:
      - download each SYMBOL,
      - compute enhanced features (parity),
      - save per-symbol Parquet to DATA_DIR,
      - write combined CSV to FEATURE_CSV,
      - return combined DataFrame.
    """
    if os.path.exists(FEATURE_CSV):
        log.info(f"Using existing features CSV: {FEATURE_CSV}")
        d = pd.read_csv(FEATURE_CSV)
        d['Datetime'] = pd.to_datetime(d['Datetime'])
        return d

    all_dfs = []
    for i, ticker in enumerate(SYMBOLS, 1):
        log.info(f"[{i}/{len(SYMBOLS)}] {ticker} — downloading")
        raw = download_stock_data(ticker, interval=INTERVAL, period_days=PERIOD_DAYS)
        if raw is None or raw.empty:
            log.warning(f"[{ticker}] no data; skipping.")
            continue
        try:
            feats, _ = compute_enhanced_features(raw)
            if feats is not None and not feats.empty:
                all_dfs.append(feats)
                out_parq = os.path.join(DATA_DIR, f"{ticker}.parquet")
                feats.to_parquet(out_parq, index=False)
                log.info(f"[{ticker}] features={len(feats)} rows → {out_parq}")
            else:
                log.warning(f"[{ticker}] empty features; skipped.")
        except Exception as e:
            log.error(f"[{ticker}] FE failed: {e}")
        finally:
            del raw
            try: del feats
            except: pass
            gc.collect()
            time.sleep(0.2)

    if not all_dfs:
        raise RuntimeError("No usable data found for any ticker.")

    full = pd.concat(all_dfs, ignore_index=True)
    full.to_csv(FEATURE_CSV, index=False)
    log.info(f"Saved combined CSV → {FEATURE_CSV} (rows={len(full)})")
    return full



In [ ]:
# --- strict determinism block (top of file) ---
import os, random

SEED = 42
os.environ["PYTHONHASHSEED"] = str(SEED)          # effective only if set in shell before launch
os.environ.setdefault("OMP_NUM_THREADS", "1")
os.environ.setdefault("MKL_NUM_THREADS", "1")
os.environ.setdefault("OPENBLAS_NUM_THREADS", "1")
os.environ.setdefault("NUMEXPR_NUM_THREADS", "1")
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":16:8"

import numpy as np            # import AFTER thread env vars
import torch                  # then torch

# pin PyTorch CPU thread pool sizes
torch.set_num_threads(1)
torch.set_num_interop_threads(1)

# seeds + determinism
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cuda.matmul.allow_tf32 = False
    torch.backends.cudnn.allow_tf32 = False

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

try:
    torch.use_deterministic_algorithms(True)
except Exception:
    pass

# disable TF32 for strict reproducibility (Ampere+)
if torch.cuda.is_available():
    torch.backends.cuda.matmul.allow_tf32 = False
    torch.backends.cudnn.allow_tf32 = False

import gc, json, sys, time, joblib, logging
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import gymnasium as gym
from gymnasium.spaces import Box
from datetime import datetime, timedelta, timezone
from stable_baselines3 import SAC
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnNoModelImprovement
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# --- deterministic setup (place near top, before importing/using CUDA) ---

# ============ Logging ============
LOG_LEVEL = os.getenv("LOG_LEVEL", "INFO").upper()
logging.basicConfig(
    level=getattr(logging, LOG_LEVEL, logging.INFO),
    format="%(asctime)s | %(levelname)s | %(name)s | %(message)s",
    handlers=[
        logging.StreamHandler(sys.stdout),
        logging.FileHandler("sac_run.log", mode="a"),
    ],
)
log = logging.getLogger("SAC-Resume")
if FAST_TEST:
    root = logging.getLogger()
    root.setLevel(logging.WARNING)
    for h in root.handlers:
        h.setLevel(logging.WARNING)

# ============ Phase 0 Toggles (wired only; no behavior change here) ============
TEST_MODE       = True
ENABLE_PLOTS    = False
ENABLE_SLO      = False
LIVE_MODE       = False
SIM_LATENCY_MS  = 0
BROKER          = "log"
USE_REGIME      = False
USE_SENTIMENT   = False
ENABLE_WAVELET  = False

# ===== Phase 2 knobs (exposure + costs) =====
MAX_EXPOSURE     = 1.0          # absolute exposure cap, e.g. 1.0 = 100% long/short
COMMISSION_BPS   = 1.0          # commission in basis points (1 bps = 0.01%)
SLIPPAGE_BPS     = 2.0          # slippage in basis points
INITIAL_CAPITAL  = 100_000.0    # starting portfolio value

# ==== Window-size probe (Phase 2.5) ====
CANDIDATE_WS   = [8, 10, 12, 16, 24]
TOP_N_WINDOWS  = 1 if TEST_MODE else 3
PROBE_STEPS    = 300 if TEST_MODE else 1500
MAX_PROBE_EVAL = 400

# Logging knobs (defaults)
LOG_TRADES_TRAIN  = False
LOG_TRADES_EVALCB = False
LOG_TRADES_FINAL  = True

# ===== Fast test run overrides (PUT THIS AFTER THE DEFAULTS) =====
FAST_TEST = True
    CANDIDATE_WS   = [10]
    TOP_N_WINDOWS  = 1
    PROBE_STEPS    = 100
    MAX_PROBE_EVAL = 150

    MIN_TRAIN_STEPS = 1000
    USE_EVAL_CALLBACK = False

    FAST["buffer_size"] = 50_000; FAST["batch_size"] = 128
    SLOW["buffer_size"] = 50_000; SLOW["batch_size"] = 128

    LOG_TRADES_FINAL = False
    os.environ["LOG_LEVEL"] = "WARNING"   # do this BEFORE logging.basicConfig
    logging.getLogger().setLevel(logging.WARNING)


# Ticker buckets
fast_names = {
    "TSLA","NVDA","AMD","AVGO","AAPL","MSFT","AMZN","GOOGL","META","ADBE","CRM",
    "INTC","QCOM","TXN","ORCL","NEE","GE","XOM","CVX","LLY","NKE","SBUX"
}
slow_names = {
    "BRK-B","JPM","BAC","JNJ","UNH","MRK","PFE","ABBV","ABT","AMGN","PG","PEP","KO",
    "V","MA","WMT","MCD","TMO","DHR","ACN","IBM","LIN","PM","RTX","UPS","UNP","COST","HD","LOW"
}

def get_sac_cfg(symbol: str) -> dict:
    if symbol in fast_names:
        bucket = "FAST"; cfg = FAST
    elif symbol in slow_names:
        bucket = "SLOW"; cfg = SLOW
    else:
        bucket = "DEFAULT"; cfg = FAST  # sensible default
    log.info("Config bucket for %s -> %s", symbol, bucket)
    return cfg | {"_bucket": bucket}

# === Latest minute data (timezone-aware 'now') ===
def latest_df_for_symbol(symbol: str, horizon_days: int = 5, interval: str = "1m"):
    end = datetime.now(timezone.utc)
    start = end - timedelta(days=horizon_days)
    df_live = yf.download(
        symbol,
        start=start.strftime("%Y-%m-%d"),
        end=end.strftime("%Y-%m-%d"),
        interval=interval,
        progress=False,
        auto_adjust=False,
    )
    if df_live is None or df_live.empty:
        log.warning("latest_df_for_symbol: no data for %s", symbol)
        return None

    df_live = df_live.reset_index()
    if "Datetime" not in df_live.columns:
        ts_col = df_live.columns[0]
        df_live = df_live.rename(columns={ts_col: "Datetime"})
    df_live["Datetime"] = pd.to_datetime(df_live["Datetime"], utc=True)
    df_live["Symbol"] = symbol

    try:
        return compute_enhanced_features(df_live)
    except Exception as e:
        log.exception("Feature engineering failed for live %s: %s", symbol, e)
        return None


log.info("Toggles | TEST_MODE=%s ENABLE_PLOTS=%s ENABLE_SLO=%s LIVE_MODE=%s SIM_LATENCY_MS=%s BROKER=%s USE_REGIME=%s USE_SENTIMENT=%s ENABLE_WAVELET=%s",
         TEST_MODE, ENABLE_PLOTS, ENABLE_SLO, LIVE_MODE, SIM_LATENCY_MS, BROKER, USE_REGIME, USE_SENTIMENT, ENABLE_WAVELET)

# ============ Colab Drive (optional) ============
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
except Exception as _:
    log.info("Colab drive not mounted (non-Colab environment).")

# ============ Paths ============
SAVE_DIR    = "/content/drive/MyDrive/Results_May_2025/results_sac_walkforward/sac_walkforward_models"
RESULTS_DIR = "/content/drive/MyDrive/Results_May_2025/results_sac_walkforward"
DATA_PATH   = "multi_stock_feature_engineered_dataset.csv"

# Ensure subdirs
for sub in ["scalers", "features", "signals", "plots", "vecnorms", "tmp"]:
    os.makedirs(os.path.join(RESULTS_DIR, sub), exist_ok=True)
os.makedirs(SAVE_DIR, exist_ok=True)

# Write a single run-level inference config (SAC) once
inference_cfg_path = os.path.join(RESULTS_DIR, "inference_config.json")
if not os.path.exists(inference_cfg_path):
    run_infer_cfg = {
        "algo": "SAC",
        "action_mode": "deterministic",
        "conf_source": "none",          # will be std|q_gap later
        "conf_thresh": None,
        "n_eval_runs": 1,
        "created_at": datetime.now().isoformat(),
    }
    with open(inference_cfg_path, "w") as f:
        json.dump(run_infer_cfg, f, indent=2)
    log.info("Wrote inference_config.json -> %s", inference_cfg_path)

# ============ Load Data ============
df = pd.read_csv(DATA_PATH)
df["Datetime"] = pd.to_datetime(df["Datetime"], utc=True)
symbols = df["Symbol"].unique()

# Completed vs remaining (resume)
completed_symbols = [
    f.split("sac_")[1].split(".")[0]
    for f in os.listdir(SAVE_DIR)
    if f.startswith("sac_") and f.endswith(".zip")
]
remaining_symbols = [s for s in symbols if s not in completed_symbols]

log.info("Completed: %d symbols | Remaining: %d symbols", len(completed_symbols), len(remaining_symbols))

# In TEST_MODE, pick 1 FAST + 1 SLOW symbol to satisfy Phase-1 gate
run_symbols = remaining_symbols
if TEST_MODE:
    fast_avail = [s for s in remaining_symbols if s in fast_names]
    slow_avail = [s for s in remaining_symbols if s in slow_names]
    pick_fast = fast_avail[0] if fast_avail else (remaining_symbols[0] if remaining_symbols else None)
    pick_slow = next((s for s in slow_avail if s != pick_fast), None) if slow_avail else None
    run_symbols = [s for s in [pick_fast, pick_slow] if s is not None]
    log.info("TEST_MODE: limiting run to (FAST, SLOW) = %s", run_symbols)

if FAST_TEST:
    run_symbols = run_symbols[:1]
    log.info("FAST_TEST: limiting to one symbol -> %s", run_symbols)

# ============ Device & Policy ============
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
policy_kwargs = dict(net_arch=[64, 64])

class ContinuousTradingEnv(gym.Env):
    """
    Phase 2: Continuous exposure env with costs.
    - Action: a in [-1,1] -> position/exposure in [-MAX_EXPOSURE, MAX_EXPOSURE]
    - Costs: commission + slippage (bps) on *position change notional*
    - Reward: portfolio return for the step
    """
    metadata = {"render_modes": []}

    def __init__(
        self,
        df: pd.DataFrame,
        frame_bound: tuple,
        window_size: int,
        max_exposure: float = MAX_EXPOSURE,
        commission_bps: float = COMMISSION_BPS,
        slippage_bps: float = SLIPPAGE_BPS,
        initial_capital: float = INITIAL_CAPITAL,
        log_trades: bool = True,
    ):
        super().__init__()
        assert "Close" in df.columns and "Datetime" in df.columns, "DataFrame must have Close & Datetime"
        self.raw_df = df.sort_values("Datetime").reset_index(drop=True).copy()
        self.window_size = int(window_size)
        self.start_tick, self.end_tick = int(frame_bound[0]), int(frame_bound[1])
        assert self.end_tick <= len(self.raw_df), "frame_bound[1] exceeds data length"
        assert self.start_tick >= self.window_size, "start tick must be >= window_size"

        # Prepare arrays
        self.prices = self.raw_df["Close"].values.astype(np.float64)
        feats = self.raw_df.drop(columns=["Symbol", "Datetime", "Close"], errors="ignore")
        feats = feats.select_dtypes(include=[np.number]).astype(np.float32).values
        self.features = feats

        # Spaces
        n_feat = self.features.shape[1]
        self.action_space = Box(low=-1.0, high=1.0, shape=(1,), dtype=np.float32)
        self.observation_space = Box(
            low=-np.inf, high=np.inf, shape=(self.window_size, n_feat), dtype=np.float32
        )

        # Trading params
        self.max_exposure = float(max_exposure)
        self.bps_total = (float(commission_bps) + float(slippage_bps)) / 10_000.0
        self.initial_capital = float(initial_capital)
        self.log_trades = bool(log_trades)
        self.log_every = 500 if TEST_MODE else 2000

        # State
        self.current_tick = None
        self.position = None         # exposure in [-max_exposure, max_exposure]
        self.portfolio_value = None
        self.last_price = None
        self.done_tick = self.end_tick - 1  # last tick we can *compute* a next return

    def _obs(self):
        start = self.current_tick - self.window_size
        obs = self.features[start:self.current_tick]
        return obs.astype(np.float32)

    def reset(self, *, seed=None, options=None):
        super().reset(seed=seed)
        self.current_tick = max(self.start_tick, self.window_size)
        self.position = 0.0
        self.portfolio_value = self.initial_capital
        self.last_price = float(self.prices[self.current_tick - 1])
        obs = self._obs()
        info = {
            "portfolio_value": float(self.portfolio_value),
            "position": float(self.position),
            "price": float(self.prices[self.current_tick]),
            "trade_cost": 0.0,
        }
        return obs, info

    def step(self, action):
        # --- Map action -> target exposure (respect cap) ---
        a = float(np.array(action).reshape(-1)[0])
        target_pos = float(np.clip(a, -1.0, 1.0)) * self.max_exposure
        target_pos = float(np.clip(target_pos, -self.max_exposure, self.max_exposure))
        old_pos = self.position

        # --- Costs on position change notional (pre-cost V) ---
        trade_notional = abs(target_pos - old_pos) * self.portfolio_value
        trade_cost = self.bps_total * trade_notional

        # Apply trade & cost immediately
        self.position = target_pos
        self.portfolio_value -= trade_cost

        # Safety: no NaNs or negatives from cost
        if not np.isfinite(self.portfolio_value) or self.portfolio_value <= 0.0:
            # Hard stop if something went off the rails
            self.portfolio_value = max(1e-8, float(self.portfolio_value))

        # --- Price move to next tick determines PnL ---
        price_t   = float(self.prices[self.current_tick])
        next_tick = self.current_tick + 1
        terminated = next_tick >= len(self.prices) or self.current_tick >= self.done_tick

        if not terminated:
            price_tp1 = float(self.prices[next_tick])
            ret = 0.0 if price_t <= 0 else (price_tp1 / price_t - 1.0)
        else:
            price_tp1 = price_t
            ret = 0.0

        # Portfolio update with exposure
        v_prev = self.portfolio_value
        self.portfolio_value = v_prev * (1.0 + self.position * ret)

        # Reward = step portfolio return (includes costs via prior deduction)
        reward = 0.0 if v_prev <= 0 else (self.portfolio_value - v_prev) / v_prev
        reward = float(np.clip(reward, -1.0, 1.0))

        # Log trade costs (only if we actually moved position)
        if self.log_trades and trade_notional > 0 and (self.current_tick % self.log_every == 0):
            log.info(
                "TRADE t=%d | price=%.4f | pos %.3f→%.3f | notional=%.2f | bps=%.2f | cost=%.2f | V: %.2f→%.2f",
                self.current_tick, price_t, old_pos, self.position,
                trade_notional, self.bps_total * 10_000.0, trade_cost,
                v_prev + trade_cost, self.portfolio_value
            )

        # Advance tick
        self.current_tick = next_tick
        obs = self._obs() if not terminated else self._obs()
        info = {
            "portfolio_value": float(self.portfolio_value),
            "position": float(self.position),
            "price": float(price_tp1),
            "trade_cost": float(trade_cost),
        }

        # No truncation logic here (you can add one by max episode len)
        truncated = False
        return obs.astype(np.float32), reward, bool(terminated), bool(truncated), info

max_ts = df["Datetime"].max()  # already tz-aware because you did utc=True above

test_end_dt    = max_ts
train_start_dt = test_end_dt - timedelta(days=729)
train_end_dt   = train_start_dt + timedelta(days=365)
test_start_dt  = train_end_dt

train_start = train_start_dt.strftime("%Y-%m-%d")
train_end   = train_end_dt.strftime("%Y-%m-%d")
test_start  = test_start_dt.strftime("%Y-%m-%d")
test_end    = test_end_dt.strftime("%Y-%m-%d")

train_start_ts = pd.to_datetime(train_start, utc=True)
train_end_ts   = pd.to_datetime(train_end,   utc=True)
test_start_ts  = pd.to_datetime(test_start,  utc=True)
test_end_ts    = pd.to_datetime(test_end,    utc=True)

log.info("Bounds UTC | train=[%s, %s) test=[%s, %s]",
         train_start_ts, train_end_ts, test_start_ts, test_end_ts)

# ============ Resume Loop ============

def _score_window_size(ws, train_df, test_df, sac_cfg, device, policy_kwargs):
    # envs for this ws
    def mk_env(df):
        return ContinuousTradingEnv(
            df, frame_bound=(ws, len(df)), window_size=ws,
            max_exposure=MAX_EXPOSURE, commission_bps=COMMISSION_BPS,
            slippage_bps=SLIPPAGE_BPS, initial_capital=INITIAL_CAPITAL,
            log_trades=False,
        )

    tr_env = DummyVecEnv([lambda: mk_env(train_df)])
    tr_env = VecNormalize(tr_env, training=True, norm_obs=True, norm_reward=False, clip_obs=10.0)
    tr_env.seed(SEED)  # ← ADD THIS

    ev_env = DummyVecEnv([lambda: mk_env(test_df)])
    ev_env = VecNormalize(ev_env, training=False, norm_obs=True, norm_reward=False, clip_obs=10.0)
    ev_env.obs_rms = tr_env.obs_rms
    ev_env.seed(SEED)  # ← ADD THIS


    # quick probe train
    m = SAC(
        "MlpPolicy", tr_env, device=device, policy_kwargs=policy_kwargs, verbose=0,
        learning_rate=sac_cfg["learning_rate"], batch_size=sac_cfg["batch_size"],
        train_freq=sac_cfg["train_freq"], gradient_steps=sac_cfg["gradient_steps"],
        gamma=sac_cfg["gamma"], tau=sac_cfg["tau"], ent_coef=sac_cfg["ent_coef"],
        buffer_size=sac_cfg["buffer_size"], learning_starts=0,
        seed=SEED,
    )
    m.learn(total_timesteps=PROBE_STEPS)

    # quick eval -> Sharpe
    obs = ev_env.reset(seed=SEED)
    pv_hist = [INITIAL_CAPITAL]
    for _ in range(min(MAX_PROBE_EVAL, len(test_df))):
        a, _ = m.predict(obs, deterministic=True)
        obs, _, done, info = ev_env.step(a)
        pv_hist.append(float(info[0].get("portfolio_value", pv_hist[-1])))
        if done[0]:
            break

    rets = pd.Series(pv_hist).pct_change().dropna()
    sharpe = float((rets.mean() / rets.std()) * np.sqrt(252)) if rets.std() and rets.std() > 0 else 0.0
    return sharpe

results = []
one_symbol_gate_done = False

for symbol in run_symbols:
    log.info("=== Resuming %s ===", symbol)
    df_sym = df[df["Symbol"] == symbol].dropna().sort_values("Datetime").reset_index(drop=True)

    train_df = df_sym[(df_sym["Datetime"] >= train_start_ts) & (df_sym["Datetime"] <  train_end_ts)].copy()
    test_df  = df_sym[(df_sym["Datetime"] >= test_start_ts)  & (df_sym["Datetime"] <= test_end_ts)].copy()

    if len(train_df) < 500 or len(test_df) < 200:
        log.warning("Skipping %s — insufficient data (train=%d, test=%d).", symbol, len(train_df), len(test_df))
        continue
    # --- Probe candidate window sizes and pick top-N ---

    # Numeric features only (Phase 0 no schema change)
    # ---- Select numeric features & scale FIRST (used by probe and final) ----
    features = [c for c in train_df.columns if c not in ["Close","Symbol","Datetime","Session"]]
    features = [c for c in features if np.issubdtype(train_df[c].dtype, np.number)]
    train_df.drop(columns=["Session"], inplace=True, errors="ignore")
    test_df.drop(columns=["Session"], inplace=True, errors="ignore")

    scaler = StandardScaler()
    train_df[features] = scaler.fit_transform(train_df[features])
    test_df[features]  = scaler.transform(test_df[features])

    joblib.dump(scaler, f"{RESULTS_DIR}/scalers/{symbol}_scaler.pkl")
    with open(f"{RESULTS_DIR}/features/{symbol}_features.json", "w") as f:
        json.dump(features, f)

    # ---- Probe candidate window sizes and pick top-N ----
    sac_cfg = get_sac_cfg(symbol)
    doc_target_update_interval = sac_cfg.get("target_update_interval")
    sac_cfg.pop("target_update_interval", None)
    log.info("Using bucket=%s | documenting target_update_interval=%s (not passed to SB3)",
            sac_cfg["_bucket"], doc_target_update_interval)
    if FAST_TEST:
        chosen_ws_list = [10]  # skip probing in fast runs
    else:
        scores = []
        for ws in CANDIDATE_WS:
            if ws >= len(train_df) or ws < 5:
                continue
            try:
                s = _score_window_size(ws, train_df, test_df, sac_cfg, device, policy_kwargs)
                scores.append((ws, s))
                log.info("WS probe %s -> Sharpe=%.3f", ws, s)
            except Exception as e:
                log.warning("WS probe failed for ws=%s: %s", ws, e)

        chosen_ws_list = [10] if not scores else [
            ws for ws, _ in sorted(scores, key=lambda x: x[1], reverse=True)[:TOP_N_WINDOWS]
        ]

    log.info("Chosen window sizes: %s", chosen_ws_list)
    WS_USE = chosen_ws_list[0]


    def make_train_env():
        return ContinuousTradingEnv(
            train_df, frame_bound=(WS_USE, len(train_df)), window_size=WS_USE,
            max_exposure=MAX_EXPOSURE, commission_bps=COMMISSION_BPS,
            slippage_bps=SLIPPAGE_BPS, initial_capital=INITIAL_CAPITAL,
            log_trades=LOG_TRADES_TRAIN,
        )

    def make_test_env_for_callback():
        return ContinuousTradingEnv(
            test_df, frame_bound=(WS_USE, len(test_df)), window_size=WS_USE,
            max_exposure=MAX_EXPOSURE, commission_bps=COMMISSION_BPS,
            slippage_bps=SLIPPAGE_BPS, initial_capital=INITIAL_CAPITAL,
            log_trades=LOG_TRADES_EVALCB,
        )

    def make_test_env_for_final():
        return ContinuousTradingEnv(
            test_df, frame_bound=(WS_USE, len(test_df)), window_size=WS_USE,
            max_exposure=MAX_EXPOSURE, commission_bps=COMMISSION_BPS,
            slippage_bps=SLIPPAGE_BPS, initial_capital=INITIAL_CAPITAL,
            log_trades=LOG_TRADES_FINAL,
        )

    train_env = DummyVecEnv([make_train_env])
    train_env = VecNormalize(train_env, training=True, norm_obs=True, norm_reward=False, clip_obs=10.0)
    train_env.seed(SEED)
    train_env.reset(seed=SEED)  # ← add

    eval_env  = DummyVecEnv([lambda: Monitor(make_test_env_for_callback())])
    eval_env  = VecNormalize(eval_env, training=False, norm_obs=True, norm_reward=False, clip_obs=10.0)
    eval_env.obs_rms = train_env.obs_rms
    eval_env.seed(SEED)
    eval_env.reset(seed=SEED)   # ← add

    # TEST_MODE: ensure learning actually starts
    if TEST_MODE:
        sac_cfg["learning_starts"] = 0
    # ---- Model ----
    model = SAC(
        "MlpPolicy",
        train_env,
        device=device,
        policy_kwargs=policy_kwargs,
        verbose=0,
        learning_rate=sac_cfg["learning_rate"],
        batch_size=sac_cfg["batch_size"],
        train_freq=sac_cfg["train_freq"],
        gradient_steps=sac_cfg["gradient_steps"],
        gamma=sac_cfg["gamma"],
        tau=sac_cfg["tau"],
        ent_coef=sac_cfg["ent_coef"],
        buffer_size=sac_cfg["buffer_size"],
        learning_starts=sac_cfg["learning_starts"],
        seed=SEED,
    )

    # --- Train schedule & callback (FAST_TEST-aware) ---
    # --- Train schedule & callback (FAST_TEST-aware) ---
    if FAST_TEST:
        eval_every  = 1_000  # ignored if callback=None
        total_steps = max(200, min(MIN_TRAIN_STEPS, len(train_df)))
        sac_cfg["learning_starts"] = 0  # start learning immediately

        callback = None if not USE_EVAL_CALLBACK else EvalCallback(
            eval_env,
            best_model_save_path=os.path.join(RESULTS_DIR, "tmp", f"best_{symbol}"),
            eval_freq=eval_every,
            n_eval_episodes=1,
            callback_after_eval=StopTrainingOnNoModelImprovement(1, 2, verbose=0),
            verbose=0,
        )
    else:
        eval_every  = 1_000 if TEST_MODE else 5_000
        total_steps = (min(4_000, len(train_df) * 2)
                      if TEST_MODE else min(25_000, len(train_df) * 10))
        if TEST_MODE:
            sac_cfg["learning_starts"] = 0

        callback = EvalCallback(
            eval_env,
            best_model_save_path=os.path.join(RESULTS_DIR, "tmp", f"best_{symbol}"),
            eval_freq=eval_every,
            n_eval_episodes=1,
            callback_after_eval=StopTrainingOnNoModelImprovement(2, 4, verbose=0),
            verbose=0,
        )

    log.info("Training %s for %d steps%s",
            symbol, total_steps, "" if callback else " (no EvalCallback)")

    t0 = time.time()
    model.learn(total_timesteps=total_steps, callback=callback)
    log.info("Training %s finished in %.1fs", symbol, time.time() - t0)

    # Save model + vecnorm
    model_path = f"{SAVE_DIR}/sac_{symbol}"
    model.save(model_path)
    vec_path = f"{RESULTS_DIR}/vecnorms/{symbol}_vecnorm.pkl"
    train_env.save(vec_path)
    log.info("Saved model: %s.zip | vecnorm: %s", model_path, vec_path)

    # Clean eval with logging of costs
    # Clean eval with logging of costs
    final_eval_raw = DummyVecEnv([make_test_env_for_final])
    final_eval_raw.seed(SEED)
    final_eval     = VecNormalize.load(vec_path, final_eval_raw)
    final_eval.training = False
    final_eval.norm_reward = False
    final_eval.obs_rms = train_env.obs_rms
    final_eval.seed(SEED)
    obs = final_eval.reset(seed=SEED)

    # Per-symbol model info shell (hyperparams + modules used)
    model_info_dir = os.path.join(RESULTS_DIR, "model_info")
    os.makedirs(model_info_dir, exist_ok=True)
    model_info = {
        "symbol": symbol,
        "bucket": sac_cfg["_bucket"],
        "algo": "SAC",
        "hyperparams": {
            "learning_rate": sac_cfg["learning_rate"],
            "batch_size": sac_cfg["batch_size"],
            "train_freq": sac_cfg["train_freq"],
            "gradient_steps": sac_cfg["gradient_steps"],
            "gamma": sac_cfg["gamma"],
            "tau": sac_cfg["tau"],
            "target_update_interval": doc_target_update_interval,
            "ent_coef": sac_cfg["ent_coef"],
            "buffer_size": sac_cfg["buffer_size"],
            "learning_starts": sac_cfg["learning_starts"],
            "policy_net_arch": policy_kwargs.get("net_arch", None),
            "device": str(device),
            "seed": SEED,
        },
        "modules": {
            "wavelet": ENABLE_WAVELET,
            "regime": USE_REGIME,
            "sentiment": USE_SENTIMENT,
            "latency_ms": SIM_LATENCY_MS,
            "commission_bps": COMMISSION_BPS,
            "slippage_bps": SLIPPAGE_BPS,
            "stop_loss": ENABLE_SLO,
            "take_profit": None,
            "cooldown": None,
            "max_exposure": MAX_EXPOSURE,
            "obs_window_sizes": chosen_ws_list,
            "broker": BROKER,
        },
        "data_window": {
            "train_start": train_start,
            "train_end": train_end,
            "test_start": test_start,
            "test_end": test_end,
        },
        "artifacts": {
            "model_zip": model_path + ".zip",
            "vecnorm_pkl": vec_path,
            "features_json": f"{RESULTS_DIR}/features/{symbol}_features.json",
            "signals_csv": f"{RESULTS_DIR}/signals/{symbol}_sac_signals.csv",
        },
        "created_at": datetime.now().isoformat(),
    }
    mi_path = os.path.join(model_info_dir, f"{symbol}_model_info.json")
    with open(mi_path, "w") as f:
        json.dump(model_info, f, indent=2)
    log.info("Wrote model_info -> %s", mi_path)

    portfolio, positions = [], []
    y_true, y_pred = [], []

    # Buy&Hold baseline
    buy_hold = float(INITIAL_CAPITAL) * (test_df["Close"].iloc[-1] / test_df["Close"].iloc[0])
    for i in range(len(test_df)):
        action, _ = model.predict(obs, deterministic=True)
        obs, _, dones, infos = final_eval.step(action)
        done = bool(dones[0])
        info0 = infos[0]
        pv = float(info0.get("portfolio_value", np.nan))
        pos = float(info0.get("position", 0.0))
        price_i = float(info0.get("price", test_df["Close"].iloc[min(i, len(test_df)-1)]))

        portfolio.append(pv)
        positions.append(pos)

        # Labels for metrics (same thresholds, but from exposure)
        pred = 1 if pos > 0.3 else -1 if pos < -0.3 else 0
        y_pred.append(pred)
        if i > 0:
            y_true.append(1 if test_df["Close"].iloc[i] > test_df["Close"].iloc[i-1]
                          else -1 if test_df["Close"].iloc[i] < test_df["Close"].iloc[i-1] else 0)
        else:
            y_true.append(0)

        if done:
            break

    aligned_len = min(len(test_df), len(portfolio))
    test_eval_df = test_df.iloc[:aligned_len].copy()
    test_eval_df["Position"] = positions[:aligned_len]
    test_eval_df["Portfolio"] = portfolio[:aligned_len]
    test_eval_df.to_csv(f"{RESULTS_DIR}/signals/{symbol}_sac_signals.csv", index=False)

    # Metrics
    try:
        accuracy  = accuracy_score(y_true, y_pred)
        precision = precision_score(y_true, y_pred, average="macro", zero_division=0)
        recall    = recall_score(y_true, y_pred, average="macro", zero_division=0)
        f1        = f1_score(y_true, y_pred, average="macro", zero_division=0)
    except Exception as e:
        log.warning("Metric calc warning on %s: %s", symbol, e)
        accuracy = precision = recall = f1 = 0.0

    final_value = portfolio[-1] if portfolio else 100_000.0
    returns = pd.Series(portfolio).pct_change().dropna()
    sharpe = (returns.mean() / returns.std()) * np.sqrt(252) if returns.std() and returns.std() > 0 else 0.0
    dd_ser  = pd.Series(portfolio)
    drawdown = float(((dd_ser.cummax() - dd_ser) / dd_ser.cummax()).max() * 100) if len(dd_ser) > 0 else 0.0
    ret = (final_value - 100_000.0) / 100_000.0

    # Plot (optional)
    if ENABLE_PLOTS:
        plt.figure(figsize=(10, 4))
        plt.plot(portfolio, label="SAC Portfolio")
        plt.axhline(y=buy_hold, linestyle="--", label="Buy & Hold")
        plt.title(f"{symbol} Portfolio vs Buy & Hold")
        plt.legend(); plt.grid(True); plt.tight_layout()
        plt.savefig(f"{RESULTS_DIR}/plots/{symbol}_portfolio.png")
        plt.close()

    results.append({
        "Ticker": symbol,
        "Model": "SAC",
        "Window": f"{train_start} to {test_end}",
        "Return": round(ret, 4),
        "SAC_Portfolio": round(final_value, 2),
        "BuyHold": round(buy_hold, 2),
        "Sharpe": round(sharpe, 3),
        "Drawdown": round(drawdown, 2),
        "Accuracy": round(accuracy, 3),
        "Precision": round(precision, 3),
        "Recall": round(recall, 3),
        "F1_Score": round(f1, 3),
        "score": round(sharpe * 0.4 + final_value * 0.000003 - drawdown * 0.3, 4),
        "Winner": "SAC" if final_value > buy_hold else "Buy & Hold",
    })

    # ---- Phase 0 Gate (run just one symbol to confirm logging + vecnorm) ----
    # ---- Phase 1 Gate: ensure we processed 2 symbols in TEST_MODE (1 FAST, 1 SLOW) ----
    if TEST_MODE:
        one_symbol_gate_done = True  # reuse flag name; now it means "we ran at least once"
        # We limited run_symbols to exactly two earlier, so no break needed here.

    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

# ============ Save Results ============
results_df = pd.DataFrame(results)
results_path = f"{RESULTS_DIR}/sac_performance_resume_summary.csv"
results_df.to_csv(results_path, index=False)
log.info("Resume run complete. Results saved: %s", results_path)